# Using Multimodal Chat with Palmyra X5

The Palmyra X5 model supports [multimodal chat](https://dev.writer.com/home/chat-with-images), letting you send both text and images in the same message. You can include local images (via data URLs) or images hosted online. This enables rich, visual conversations directly in chat completions.


## Prerequisites

Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/api-guides/quickstart)

## Setup
Install the Writer SDK:

In [ ]:
%pip install -qU writer-sdk

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

Multimodal chat uses mixed content support, where the `content` field in a message can be an array of content fragments. Each fragment can be either text or an image:

```json
{
  "role": "user",
  "content": [
    {
      "type": "text",
      "text": "Your text content here",
    },
    {
      "type": "image_url",
      "image_url": {
        "url": "https://example.com/image.jpg"
      }
    }
  ]
}
```

The `content` array can contain any combination of text (type: `"text"`) and image URLs (type: `"image_url"`).

Each `image_url` can point to a hosted image or a local image converted to a base64 data URL.

This snippet lets you define the text and images you want to send in a multimodal chat. The system constructs the inputs array with this information and passes it to the LLM for a contextual response:

In [ ]:
text_input = "What do you see in this image?"

image_input = "https://ichef.bbci.co.uk/news/1024/cpsprodpb/168F6/production/_130660429_1977_august_september_voyager1_1.jpg.webp"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_input
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)


## Using local images with data URLs
You can also use [data URLs](https://developer.mozilla.org/en-US/docs/Web/URI/Reference/Schemes/data) to send local images without uploading them to a remote server. This is useful for testing or when working with local files:

In [ ]:
import base64
from pathlib import Path

# Read local image and convert to base64 data URL
def image_to_data_url(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
        file_extension = Path(image_path).suffix.lstrip('.')
        mime_type = f"image/{file_extension}" if file_extension in ['jpg', 'jpeg', 'png', 'gif'] else "image/jpeg"
        return f"data:{mime_type};base64,{encoded_string}"

# Convert local image to data URL
notebook_dir = os.path.dirname(os.path.abspath("multimodal-chat-x5.ipynb"))
image_path = os.path.join(notebook_dir, "media", "image.jpg")
data_url = image_to_data_url(image_path)

text_input = "What do you see in this image?"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": data_url
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)

## Multiple images with text
You can include multiple images in a single message:

In [ ]:
text_input = "Compare these two images and tell me the differences:"
image_for_comparison = "https://hips.hearstapps.com/hmg-prod/images/o5i2rc7u3wbj4f2ybg6zyd-1657151459.jpg"

response = client.chat.chat(
  model="palmyra-x5",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_input
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_input
          }
        },
        {
          "type": "image_url",
          "image_url": {
            "url": image_for_comparison
          }
        }
      ]
    }
  ]
)

print(response.choices[0].message.content)